In [1]:
from euriai.langchain import create_chat_model

chat_model = create_chat_model(
    api_key="euri-1f3025f0ddab44e290bf86bed48790c9f4fd1325a59b42e49701607ebb5b9546",
    model="gpt-4.1-nano",
    temperature=0.7
)

response = chat_model.invoke("What is artificial intelligence?")
print(response.content)


Artificial intelligence (AI) refers to the simulation of human intelligence processes by machines, particularly computer systems. These processes include learning (the ability to improve performance over time), reasoning (the ability to analyze information and make decisions), problem-solving, understanding natural language, perception (such as vision and speech recognition), and even creativity. AI systems can range from simple algorithms performing specific tasks to complex models like deep learning networks that can analyze vast amounts of data and adapt their behavior accordingly. The goal of AI is to create systems that can perform tasks typically requiring human intelligence, ultimately enabling automation, improved decision-making, and new technological capabilities across various fields.


In [2]:
import pydantic
print(pydantic.__version__)

2.11.7


In [ ]:
pip show euriai

In [3]:
pip install --upgrade euriai


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from euriai.langchain import create_chat_model

# Full streaming support
chat_model = create_chat_model(api_key="euri-1f3025f0ddab44e290bf86bed48790c9f4fd1325a59b42e49701607ebb5b9546")

# Streaming response
for chunk in chat_model.stream("Tell me a story about god Shri Krishna"):
    print(chunk.content, end="", flush=True)

# Async streaming
import asyncio

async def async_stream():
    async for chunk in chat_model.astream("Tell me a story about god Shri Krishna"):
        print(chunk.content, end="", flush=True)

# asyncio.run(async_stream())

In [ ]:
pip install -U langchain langchain-core langchain-community langchain-openai

In [2]:
import ast,re,math,os,sys
os.environ["euri-api-key"] ="euri-1f3025f0ddab44e290bf86bed48790c9f4fd1325a59b42e49701607ebb5b9546"
from euriai.langchain import create_chat_model
from langchain.agents import create_react_agent,AgentExecutor
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate,MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough,RunnableLambda,RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory ,InMemoryChatMessageHistory as ChatMessageHistory
from langchain_core.tools import BaseTool,tool
from langchain.memory import (
    ConversationBufferMemory,
    ConversationBufferWindowMemory,
    ConversationSummaryMemory,
    CombinedMemory,
    ReadOnlySharedMemory,
    ConversationEntityMemory

)


In [3]:


from euriai.langchain import create_chat_model

llm_default = create_chat_model(
    api_key="euri-1f3025f0ddab44e290bf86bed48790c9f4fd1325a59b42e49701607ebb5b9546",
    model="gpt-4.1-nano",
    temperature=0.7
)


In [4]:
@tool("calculator",return_direct=True)
def calculator(expression: str) -> str:
    """Evaluate a numeric math expression . supprts + , -,*,/,**,parentheses and all kind of mathamatical functions"""
    allowed_nodes =(
        ast.Expression,
        ast.UnaryOp,
        ast.unaryop,
        ast.BinOp,
        ast.operator,
        ast.Num,
        ast.Load,
        ast.pow,
        ast.FunctionDef,
        ast.Module,
        ast.Expr,
        ast.Call,
        ast.Name,
        ast.arguments,
        ast.args,
        ast.Constant
    )
    
    allowed_names = {k:v for k,v in vars(math).items() if not k.startswith("_")}
    allowed_names.update({"abs": abs, "round": round,"min": min,"max": max})
    node = ast.parse(expression, mode="eval")
    
    
    for n in ast.walk(node):
        if not isinstance(n, allowed_nodes):
            raise ValueError(f"Expression contains invalid node {type(n)}")
        if isinstance(n, ast.Name) and n.id not in allowed_names:
            raise ValueError(f"Expression contains invalid name {n.id}")
    code = compile(node, "<string>", "eval")
    return str(eval(code, {"__builtins__": {}}, allowed_names))

In [5]:
tools_math = [calculator]

In [6]:
react_prompt_math = ChatPromptTemplate.from_messages([(
    "system","you are a precise math assistant , you can use thse tools:\n{tools}\n"
    "when you are going to use this tools follw these instruction exactly the same format:\n"
    "Questions :......\nthought ...\nAction by using one of the tools access that you have [{tool_names}]\n "
    "alwasy finish with final give me a numeric answer to the question"),
    ("human","questions: {input}\n{agent_scratchpad}")
])

In [7]:
agent_math = create_react_agent(
    llm=llm_default,
    prompt=react_prompt_math,
    tools=tools_math
)

In [8]:
memory_math = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    input_key="input",
    output_key="output",
)

C:\Users\u730804\AppData\Local\Temp\ipykernel_11088\1516692622.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory_math = ConversationBufferMemory(


In [9]:
math_exector = AgentExecutor(
    agent=agent_math,
    tools=tools_math,
    memory=memory_math,
    verbose=True,
    handle_parsing_errors=(
        "you did not follow the instruction that i have given you to use the tools"
    ),
)

In [10]:
@tool("kb_search", return_direct=True)
def kb_search(query: str) -> str:
    """a mock function to search from a knowledge base"""
    knowledge_base = {
        "what is the capital of france": "The capital of France is Paris.",
        "who is the president of the united states": "The president of the United States is Joe Biden.",
        "what is the largest mammal": "The largest mammal is the blue whale.",
    }
    query = query.lower().strip("?")
    for key,answer in knowledge_base.items():
        if key in query:
            return answer
    return "I don't know the answer to that question."

In [11]:
tools_kb = [kb_search]

In [12]:
react_prompt_kb = ChatPromptTemplate.from_messages([
    ("system","you are a helpful assistant that can answer question based on your knowledge base and you can use the following tools:\n{tools}\n"
    "when you are going to use this tools follw these instruction exactly the same format:\n"
    "Questions :......\nthought ...\nAction by using one of the tools access that you have [{tool_names}]\n "
    "alwasy finish with final answer to the question that you are able to search from my knowledge base"),
    ("human","questions: {input}\n{agent_scratchpad}")
])

In [13]:
agent_kb = create_react_agent(
    llm=llm_default,
    prompt=react_prompt_kb,
    tools=tools_kb
)

In [14]:
mem_kb = ConversationBufferWindowMemory(
    memory_key="chat_history",
    return_messages=True,
    input_key="input",
    output_key="output",
    k=5
)

C:\Users\u730804\AppData\Local\Temp\ipykernel_11088\4073518797.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  mem_kb = ConversationBufferWindowMemory(


In [15]:
kb_executor = AgentExecutor(
    agent=agent_kb,
    tools=tools_kb,
    memory=mem_kb,
    verbose=True,
    handle_parsing_errors=(
        "you did not follow the instruction that i have given you to use the kb  tools"
    ),
)

In [16]:
"""Steps to create Agent:
1. Create LLM first 
2. Write your function convert it into @Tool.
3. Create instruction PromptTemplate for your Agent
4. Create Agent using create_react_agent and tools and PromptTemplate as input
4. add memory for your agents
5. final create AgentExecutor using agent,tools and memory as input
"""

'Steps to create Agent:\n1. Create LLM first \n2. Write your function convert it into @Tool.\n3. Create instruction PromptTemplate for your Agent\n4. Create Agent using create_react_agent and tools and PromptTemplate as input\n4. add memory for your agents\n5. final create AgentExecutor using agent,tools and memory as input\n'

In [17]:
router_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a router. Read the user message and output exactly one token:\n"
     "- MATH: if it asks for calculations.\n"
     "- KB: otherwise (tech education, RAG, agents, general know-how).\n"
     "Output only MATH or KB."),
    ("human", "{input}")
])

In [24]:
router_chain = router_prompt|llm_default|StrOutputParser()

In [25]:
def _dispatcher(inputs:dict):
    """it will take new inputs from human and it will also attach hitory from memory """
    user_msg = inputs["input"]
    choice = router_chain.invoke({"input": user_msg}).strip().upper()
    print(f"router choice : {choice}")
    if choice == "MATH" or choice == "Math" or choice == "math":
        return math_exector.invoke({"input": user_msg})
    elif choice == "KB" or choice == "Kb" or choice == "kb":
        return kb_executor.invoke({"input": user_msg})
    else:
        return "I can only answer math and kb related questions"

In [26]:
dispatcher = RunnableLambda(_dispatcher)

In [27]:
_sessions = {}

In [28]:
def _get_history(session_id:str):
    if session_id not in _sessions:
        _sessions[session_id] = ChatMessageHistory()
    return _sessions[session_id]

In [29]:
orchestrator = RunnableWithMessageHistory(
    runnable=dispatcher,
    get_session_history=_get_history,
    input_key="input",
    history_messages_key="history"
)

In [30]:
cfg = {"configurable":{"session_id":"user1"}}

In [31]:
print(orchestrator.invoke({"input":"what is the capital of france?"},config=cfg))

router choice : KB


> Entering new AgentExecutor chain...
Could not parse LLM output: `Thought: I need to find the capital of France from the knowledge base.  
Action by using one of the tools: [kb_search]  
query: What is the capital of France?`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE you did not follow the instruction that i have given you to use the kb  toolsCould not parse LLM output: `Thought: The question is about the capital of France. Since I know from my training data that the capital of France is Paris, I can provide that answer directly.  
Final answer: The capital of France is Paris.`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE you did not follow the instruction that i have given you to use the kb  toolsCould not parse LLM output: `Thought: The question asks for the capital of France. Based on my knowledge, the capital of France is Paris. Sinc

In [32]:
print(orchestrator.invoke({"input":"what is sin(90)"},config=cfg))

router choice : MATH


> Entering new AgentExecutor chain...
Could not parse LLM output: `Thought: I need to calculate the sine of 90 degrees.

Action by using one of the tools access that you have [calculator]
calculator("sin(90)")`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE you did not follow the instruction that i have given you to use the toolsCould not parse LLM output: `Questions: what is sin(90)

Thought: I need to calculate the sine of 90 degrees, but I should ensure the calculator function uses radians if necessary. Since the calculator likely uses radians, I will convert 90 degrees to radians first.

Conversion: \( 90^\circ = \frac{\pi}{2} \)

Action by using one of the tools access that you have [calculator]
calculator("sin(pi/2)")`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE you did not follow the instruction that i have given you to use the tools

In [ ]:
1.whatsapp tool
2.email tool
3.sms tool
4.telegram tool
5.calander tool